In [1]:
from binance.client import Client
from datetime import datetime, timezone
import csv

# Claves API de tu cuenta mock
api_key = '7de4ca57d53aac719f84f4f2f1ceee269512766e19734dca3beaec464c92a2d1'
api_secret = '15122b41c53e6293bd122333bf8f31a12e7f8b876595a290473487e38a25d3f0'

# Conexión con Binance Futures testnet
client = Client(api_key, api_secret, testnet=True)
client.FUTURES_URL = "https://testnet.binancefuture.com/fapi/v1"

In [5]:
def list_available_pairs():
    """List all available trading pairs on Binance Futures."""
    exchange_info = client.futures_exchange_info()
    symbols = [symbol['symbol'] for symbol in exchange_info['symbols']]
    return symbols

# Ejecuta para listar pares
available_pairs = list_available_pairs()

print(f"Available pairs: {available_pairs}")


Available pairs: ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT', 'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT', 'COMPUSDT', 'OMGUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'BALUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'ENJUSDT', 'FLMUSDT', 'TOMOUSDT', 'RENUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'OCEANUSDT', 'BELUSDT', 'CTKUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', 'BNTUSDT', '1INCHUSDT', 'UNFIUSDT', 'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'LITUSDT', 'REEFUSDT', 'RVNUSDT', 'SFPUSDT', 'XEMUSDT', 'BTCSTUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSD

In [6]:
def get_current_prices(pairs):
    """Fetch the current prices for the specified trading pairs."""
    prices = {}
    for pair in pairs:
        try:
            ticker = client.futures_symbol_ticker(symbol=pair)
            prices[pair] = float(ticker['price'])
        except Exception as e:
            print(f"Error fetching price for {pair}: {e}")
    return prices

# Especifica los pares
pairs_to_check = ["BTCUSDT", "ETHUSDT", "BNBUSDT"]

# Ejecuta para obtener precios actuales
current_prices = get_current_prices(pairs_to_check)

print("Current prices:")

for pair, price in current_prices.items():
    print(f"{pair}: {price}")


Current prices:
BTCUSDT: 96464.2
ETHUSDT: 3232.48
BNBUSDT: 706.779


In [11]:
def get_historical_prices(symbol, interval, start_time=None, end_time=None, limit=100):
    """
    Fetch historical price data (candlesticks) for a given trading pair.
    """
    try:
        if isinstance(start_time, str):
            start_time = int(datetime.strptime(start_time, "%Y-%m-%d").timestamp() * 1000)
        if isinstance(end_time, str):
            end_time = int(datetime.strptime(end_time, "%Y-%m-%d").timestamp() * 1000)

        klines = client.futures_klines(
            symbol=symbol,
            interval=interval,
            startTime=start_time,
            endTime=end_time,
            limit=limit
        )
        return klines
    except Exception as e:
        print(f"Error fetching historical prices for {symbol}: {e}")
        return []

# Obtén datos históricos de BTC
btc_historical = get_historical_prices("BTCUSDT", "5m", limit=100)
btc_historical[:5]  # Muestra las primeras 5 filas


[[1736877300000,
  '95990.40',
  '95998.00',
  '95990.40',
  '95997.90',
  '0.655',
  1736877599999,
  '62877.05370',
  62,
  '0.381',
  '36575.21910',
  '0'],
 [1736877600000,
  '95997.90',
  '97198.00',
  '95997.90',
  '96686.00',
  '7.105',
  1736877899999,
  '686014.44980',
  170,
  '4.811',
  '464275.01770',
  '0'],
 [1736877900000,
  '96699.80',
  '96699.90',
  '96686.00',
  '96686.10',
  '1.593',
  1736878199999,
  '154028.30650',
  24,
  '0.540',
  '52217.93600',
  '0'],
 [1736878200000,
  '96686.10',
  '97200.00',
  '96686.00',
  '96686.10',
  '6.502',
  1736878499999,
  '629399.24130',
  65,
  '5.601',
  '542284.99330',
  '0'],
 [1736878500000,
  '97198.00',
  '97279.00',
  '96425.10',
  '97279.00',
  '21.792',
  1736878799999,
  '2113035.73600',
  151,
  '10.825',
  '1051532.86730',
  '0']]

In [12]:
def export_to_csv(klines, symbol, interval):
    """
    Export kline (candlestick) data to a CSV file with a dynamic filename.
    
    The filename includes the symbol, interval, and current date to avoid overwriting.
    """
    # Generar el nombre del archivo dinámicamente
    current_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')  # Fecha y hora actual
    filename = f"data/{symbol}_{interval}_{current_date}.csv"
    
    headers = [
        "Open Time", "Open", "High", "Low", "Close", "Volume",
        "Close Time", "Quote Asset Volume", "Number of Trades",
        "Taker Buy Base Volume", "Taker Buy Quote Volume", "Ignore"
    ]
    with open(filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write the header row
        for kline in klines:
            kline[0] = datetime.fromtimestamp(kline[0] / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
            kline[6] = datetime.fromtimestamp(kline[6] / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow(kline)  # Write each kline as a row
    
    print(f"Historical data exported to {filename}")

# Exporta los datos históricos de BTC a un archivo CSV
export_to_csv(btc_historical, "BTCUSDT", "5m")


Historical data exported to data/BTCUSDT_5m_2025-01-14_21-13-42.csv
